In [92]:
# Titanic Data

import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt 

df = pd.read_csv('./data/titanic/train.csv')
#display(df) # 891 rows × 12 columns

# 데이터 전처리 
# 문자는 처리 못함
# id 는 그대로 - 생존여부와 상관이 있나?! - 없음 - 종속변수에 영향을 미치지 않음
# Survived - 종속변수 (0,1)
# Pclass - 영향이 많음
# Name - 이름 자체는 생존여부에 영향을 미치지 않음
# Sex - 생존여부에 영향이 많음 - 문자로 되어있음 - 숫자로 변환
# Age - 결측지가 있는데 삭제 보단 대체값 사용
#     - 전체평균? 다른방식? 많이 사용하는 방식은 이름 Mr,Miss, Mrs 의 평균 사용
#     - 0~ 10 소아 (0), 10~25: 청년 (1), 26~~~ 범위값으로 숫자로 mapping
# SibSp	
# Parch
# Ticket - 포함 x
# Fare - ?
# Cabin - Nan을 수정해서 사용할지 결정
# Embarked - 포함

# 필요없는 속성 삭제
df = df.drop(['PassengerId','Name','SibSp','Parch','Ticket','Fare','Cabin'], axis=1)

# sex를 문자열에서 숫자로 변경
df['Sex']= df['Sex'].map({'male': 0, 'female': 1})

# age 열의 nan을 전체 평균으로 대체
avg_age = round(df['Age'].mean())
df['Age'] = df['Age'].replace(np.nan, avg_age)
# age mapping
df.loc[(df.Age < 20) , 'Age'] = 0
df.loc[((df.Age >= 20) & (df.Age < 40)) , 'Age'] = 1
df.loc[((df.Age >= 40) & (df.Age < 65)) , 'Age'] = 2
df.loc[(df.Age >= 65) , 'Age'] = 3
#display(df)

# Embarked 문자에서 숫자로 변경, 결측치 삭제
df['Embarked']= df['Embarked'].map({'S':0,'C':1,'Q':2})
df = df.dropna(how='any') # 889 rows × 6 columns

display(df)
display(df.isnull().sum())

# # 이상치 점검
# fig = plt.figure()
# fig_Pclass = fig.add_subplot(1,4,1)
# fig_Sex = fig.add_subplot(1,4,2)
# fig_Age = fig.add_subplot(1,4,3)
# fig_Embarked = fig.add_subplot(1,4,4)
# fig.tight_layout()
# fig_Pclass.boxplot(df['Pclass'])
# fig_Sex.boxplot(df['Sex'])
# fig_Age.boxplot(df['Age'])
# fig_Embarked.boxplot(df['Embarked'])

# # 나이는 그냥 가져가도록 함

x_data = df.drop('Survived', axis=1, inplace=False).values
t_data = df['Survived'].values.reshape(-1,1)


,Survived,Pclass,Sex,Age,Embarked
0,0,3,0,1.0,0.0
1,1,1,1,1.0,1.0
2,1,3,1,1.0,0.0
3,1,1,1,1.0,0.0
4,0,3,0,1.0,0.0
...,...,...,...,...,...
886,0,2,0,1.0,0.0
887,1,1,1,0.0,0.0
888,0,3,1,1.0,0.0
889,1,1,0,1.0,1.0


Survived    0
Pclass      0
Sex         0
Age         0
Embarked    0
dtype: int64

In [106]:
# tensorflow
import tensorflow as tf

X = tf.placeholder(shape=[None,4], dtype=tf.float32)
T = tf.placeholder(shape=[None,1], dtype=tf.float32)

W = tf.Variable(tf.random.normal([4,1]), name='weight')
b = tf.Variable(tf.random.normal([1]), name='bias')

logit = tf.matmul(X,W) + b
H = tf.sigmoid(logit)

loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=logit, labels=T))

train = tf.train.GradientDescentOptimizer(learning_rate=1e-3).minimize(loss)

sess = tf.Session()
sess.run(tf.global_variables_initializer())

for step in range(30000):
    _, W_val, b_val, loss_val = sess.run([train,W,b,loss], feed_dict={X:x_data, T:t_data})
    
    if step % 3000 == 0:
        print(W_val.ravel(),b_val,loss_val)


[-0.24519281 -1.4351491  -1.444466    1.1662816 ] [0.0725743] 1.0571404
[-0.09902073 -0.88116723 -1.0257225   1.1253753 ] [0.3557806] 0.8097377
[-0.25967273 -0.42405683 -0.793947    1.0099609 ] [0.47873753] 0.70374626
[-0.3912052  -0.030165   -0.63258445  0.89684963] [0.5735601] 0.6300531
[-0.49275744  0.30557764 -0.52612466  0.7958327 ] [0.6470457] 0.57987577
[-0.57061416  0.5907969  -0.45939884  0.7103897 ] [0.70440555] 0.5456076
[-0.63094634  0.8335875  -0.41966137  0.64024866] [0.7500231] 0.52181417
[-0.6786678  1.0411975 -0.3975537  0.5834714] [0.7872193] 0.5049451
[-0.71733654  1.2196678  -0.38667855  0.537736  ] [0.8183796] 0.49274188
[-0.74942744  1.3738996  -0.38282683  0.5008896 ] [0.8451809] 0.4837545


In [135]:
# predict data 전처리
predict_df = pd.read_csv('./data/titanic/test.csv')
pass_id = predict_df['PassengerId']
predict_df = predict_df.drop(['PassengerId','Name','SibSp','Parch','Ticket','Fare','Cabin'], axis=1)
predict_df['Sex']= predict_df['Sex'].map({'male': 0, 'female': 1})

# age 열의 nan을 전체 평균으로 대체
avg_age = round(predict_df['Age'].mean())
predict_df['Age'] = predict_df['Age'].replace(np.nan, avg_age)
# age mapping
predict_df.loc[(predict_df.Age < 20) , 'Age'] = 0
predict_df.loc[((predict_df.Age >= 20) & (predict_df.Age < 40)) , 'Age'] = 1
predict_df.loc[((predict_df.Age >= 40) & (predict_df.Age < 65)) , 'Age'] = 2
predict_df.loc[(predict_df.Age >= 65) , 'Age'] = 3

predict_df['Embarked']= predict_df['Embarked'].map({'S':0,'C':1,'Q':2})
predict_df = predict_df.dropna(how='any') # 889 rows × 6 columns

predict_data = predict_df.values
print(predict_data)

[[3. 0. 1. 2.]
 [3. 1. 2. 0.]
 [2. 0. 2. 2.]
 ...
 [3. 0. 1. 0.]
 [3. 0. 1. 0.]
 [3. 0. 1. 1.]]


In [168]:
result = sess.run(H, feed_dict={X:predict_data})

result_df = pd.DataFrame(result, columns=['result'])
result_df['PassengerId'] = pass_id
result_df.loc[result_df.result <0.5, 'Survived'] = '0'
result_df.loc[result_df.result >= 0.5, 'Survived'] = '1'
result_df_1 = result_df.drop(['result'], axis=1, inplace=False)
print(result_df_1)

result_df_1.to_csv('./data/titanic/predict_result.csv', header=True, index=False)

     PassengerId Survived
0            892        0
1            893        0
2            894        0
3            895        0
4            896        0
..           ...      ...
413         1305        0
414         1306        1
415         1307        0
416         1308        0
417         1309        0

[418 rows x 2 columns]


In [ ]:
# 77% 맞음